### Motivating Recurrent Neural Networks
Picking up the discussion from where we left off, we proposed a model over 2 variables $x_1, x_2$, learning a distribution $p(x_1, x_2)$ as $p(x_1) \cdot p(x_2 ~|~ x_1)$, where 
- $p(x_1)$ is estimated using a histogram
- $p(x_2 ~|~ x_1)$ is estimated using a MLP

The conclusion being that such a model is bad because
- there is one function approximator per conditional
- no information is shared amoungst different conditionals

The solution to both problems being share parameters between a single function approximator

### RNN Review

What is an RNN?
```python
def rnn_step(self, input, hidden):
    # if we simply had
    # hidden = torch.tanh(input @ self.Wxh + self.bh)
    # it would just be a multi layer perceptron

    # by adding hidden @ self.Whh we using the context of the previous
    # hidden state
    hidden = input @ self.Wxh +  hidden @ self.Whh + self.bh
    output = hidden @ self.Why + self.by

    output = self.softmax(output)

    return output, hidden
```
The comment really covers it, an RNN distinguishes itself from a MLP in that with each step we are not only updating the weights of previous layers, but carrying forward the hidden state from past iterations. To see in more detail how the LSTM improves on the existing RNN model, see the nlp section.

### Imports

In [1]:
# data processing
import numpy as np
from tqdm import tqdm

# Torch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

# Torchvision Imports
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
# GPU acceleration
gpu = torch.device("cuda:0")

### Dataset

In [ ]:
MNIST_SIZE = 28
trans = transforms.Compose([
                            transforms.Grayscale(), 
                            transforms.Resize((MNIST_SIZE, MNIST_SIZE)), 
                            transforms.ToTensor()
])
mnist_loader = torch.utils.data.DataLoader(datasets.MNIST(root='./data', train=True, download=True, 
                                                          transform=trans), shuffle=True)

### Architecture Overview
RNN combined with MLP

In [3]:
class RNN_Example(nn.Module):

    def __init__(self, device, append_loc=False, input_shape=(1, 28, 28), hidden_size=256):
        super().__init__()
        self.device = device
        self.append_loc = append_loc
        self.input_channels = input_shape[0] + 2 if append_loc else input_shape[0]
        self.hidden_size = hidden_size
        self.input_shape = input_shape
        self.canvas_size = input_shape[1] * input_shape[2]

        self.lstm = nn.LSTM(self.input_channels, self.hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, input_shape[0])

    def loss(self, x):
        batch_size = x.shape[0]
        x_inp = append_location(x, self.device) if self.append_loc else x

        # Shift input by one to the right
        x_inp = x_inp.permute(0, 2, 3, 1).contiguous().view(batch_size, self.canvas_size, self.input_channels)
        x_inp = torch.cat((torch.zeros(batch_size, 1, self.input_channels).to(self.device), x_inp[:, :-1]), dim=1)

        h0 = torch.zeros(1, x_inp.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(1, x_inp.size(0), self.hidden_size).to(self.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x_inp, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out).squeeze(-1) # b x 784

        return F.binary_cross_entropy_with_logits(out, x.view(batch_size, -1))


    def sample(self, n):
        with torch.no_grad():
            samples = torch.zeros(n, 1, self.input_channels).to(self.device)
            h = torch.zeros(1, n, self.hidden_size).to(self.device)
            c = torch.zeros(1, n, self.hidden_size).to(self.device)

            for i in range(self.canvas_size):
                x_inp = samples[:, [i]]
                out, (h, c) = self.lstm(x_inp, (h, c))
                out = self.fc(out[:, 0, :])
                prob = torch.sigmoid(out)
                sample_pixel = torch.bernoulli(prob).unsqueeze(-1) # n x 1 x 1
                if self.append_loc:
                    loc = np.array([i // 28, i % 28]) / 27
                    loc = torch.FloatTensor(loc).to(self.device)
                    loc = loc.view(1, 1, 2).repeat(n, 1, 1)
                    sample_pixel = torch.cat((sample_pixel, loc), dim=-1)
                samples = torch.cat((samples, sample_pixel), dim=1)

            if self.append_loc:
                samples = samples[:, 1:, 0] # only get sampled pixels, ignore location
            else:
                samples = samples[:, 1:].squeeze(-1) # n x 784
            samples = samples.view(n, *self.input_shape)
            return samples.cpu()

### Training

In [ ]:
def train(train_loader, d, num_epochs=100):
    model = SimpleAutoregModel(d)
    model = model.to(device)
    
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    for epoch in tqdm(range(num_epochs), desc='Training model'):  # loop over the dataset multiple times
        for i, data in enumerate(train_loader):
            x = data.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            loss = model.loss(x)
            loss.backward()
            optimizer.step()
            
            
    return model